# Boston Earnings
This notebook builds off Boston Pensions Pt 1. Instead of just looking at cop pay in this data, I will use it to find cop pay as % of total pay for the pension analysis and hopefully ultimately for our final calculation of what the Boston PD costs. 

This dataset has in it total payments to boston cops over 2016-2020 that's great than the total stated budget of the Boston PD. There are no columns for "pension" in this data, so current assumption is that discrepancy is due to what we vera called "fringe" benefits in https://www.vera.org/downloads/publications/a-look-inside-the-new-york-city-police-department-budget.pdf

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# from Boston_Earnings_Helpers import get_numeric, string_to_float

In [3]:
to_float_cols = ["REGULAR", "RETRO", "OTHER", "OVERTIME", 
                 "INJURED", "DETAIL","QUINN/EDUCATION INCENTIVE", "TOTAL EARNINGS"]

First thing to check is if we got all the data

In [4]:
boston_earnings_2019 = pd.read_csv("../data/BostonPD/Boston_Earnings_2019.csv")



In [71]:
def string_to_float(val):
    if isinstance(val, int) or isinstance(val, float):
        return val
    elif "-" in val:
        return 0
    else:
        return to_numeric(val)

def to_numeric(val):
    for c in [" ", ",", "(", ")"]:
        val = val.replace(c, "")
    try:
        return float(val)
    except:
        return 0

In [72]:
boston_earnings_2019.head(5)

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS,POSTAL,year
0,"Abadi,Kidani A",Assessing Department,Property Utilization Officer,70738.870,0.000,275.000,0.000,0.000,0.000,0.000,71013.870,2118,2019
1,"Abajue Umeh,Kenechukwu Stephanie",BPS Boston Arts Academy,Teacher,67296.000,1290.630,500.000,0.000,0.000,0.000,0.000,69086.630,2150,2019
2,"Abasciano,Joseph",Boston Police Department,Police Officer,94156.490,0.000,850.000,26496.590,0.000,5980.000,23539.210,151022.290,2132,2019
3,"Abatzis,Kelley",BPS Health and Wellness,Employment Agreement,5880.000,0.000,0.000,0.000,0.000,0.000,0.000,5880.000,2171,2019
4,"Abban,Christopher John",Boston Fire Department,Fire Fighter,118315.560,8766.010,3680.980,18437.270,2279.780,1944.000,0.000,153423.600,2132,2019


In [73]:
boston_earnings_2019[to_float_cols] = boston_earnings_2019[to_float_cols].applymap(string_to_float)
boston_earnings_2019["year"] = 2019

In [74]:
boston_earnings_2019["INJURED"].sum()

39958682.77

In [75]:
boston_earnings = boston_earnings_2019
for year in (2018, 2017, 2016):
    print("got to year", year)
    earnings = pd.read_csv("../data/BostonPD/Boston_Earnings_" + str(year) + ".csv", engine="python")
    earnings["year"] = year
    if year == 2017:
        earnings.rename(columns = {"DEPARTMENT NAME":"DEPARTMENT_NAME"}, inplace=True)
    earnings[to_float_cols] = earnings[to_float_cols].applymap(string_to_float)
    boston_earnings = boston_earnings.append(earnings)
boston_earnings.shape

got to year 2018
got to year 2017
got to year 2016


(91208, 13)

In [79]:
boston_earnings.groupby("year").sum()

,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS
year,,,,,,,,
2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2017,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2018,1416825526.630,29946840.600,52691623.900,120139913.710,33104131.130,42936237.430,24947885.890,1720567517.370
2019,1470657000.870,34122856.390,55433468.960,127017515.860,39958682.770,40698940.490,28871146.820,1796738085.860


,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS,POSTAL


In [37]:
boston_earnings.groupby("year").size()

year
2016    22046
2017    22245
2018    23605
2019    23312
dtype: int64

In [38]:
pd.set_option("display.max.rows", 50)
boston_earnings.groupby("DEPARTMENT_NAME").sum()[to_float_cols].sort_values("TOTAL EARNINGS", ascending=False).head(50)

,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS
DEPARTMENT_NAME,,,,,,,,
Boston Police Department,939454359.950,37678617.690,59306315.890,283560824.630,55878351.840,143621695.000,92865300.880,1612273457.620
Boston Fire Department,690720960.060,17635020.570,28619090.430,100547361.420,43860829.400,30719115.860,13084.310,912059861.490
BPS Special Education,180868335.850,2169903.360,3863371.720,86694.920,92770.960,0.000,0.000,187081076.810
BPS Facility Management,97960341.080,1275581.430,2855488.920,8006991.630,1143738.940,0.000,0.000,111241707.200
Boston Public Library,103189114.150,1757227.990,2121524.780,2737974.750,100169.260,0.000,0.000,109906010.930
Public Works Department,83572033.550,377135.530,1640633.400,19038651.110,1330686.820,0.000,0.000,105959140.410
Boston Cntr - Youth & Families,87047918.880,540181.180,1410248.990,1190382.790,40860.130,0.000,0.000,90229591.970
Traffic Division,74010454.400,291004.140,1396474.880,6792476.720,1291974.570,0.000,0.000,83782384.710
Inspectional Services Dept,58817112.210,328189.930,2063774.520,6321548.990,146396.280,1148.000,0.000,67676174.850


In [39]:
bostonPD_by_year = boston_earnings[boston_earnings["DEPARTMENT_NAME"]=="Boston Police Department"]\
                    .groupby("year").sum()["TOTAL EARNINGS"]
total_by_year = boston_earnings.groupby("year").sum()["TOTAL EARNINGS"]
total_by_year

year
2016   1580210286.370
2017   1590905618.100
2018   1720567517.370
2019   1796738085.860
Name: TOTAL EARNINGS, dtype: float64

In [40]:
bostonPD_by_year/total_by_year

year
2016   0.246
2017   0.246
2018   0.242
2019   0.231
Name: TOTAL EARNINGS, dtype: float64

In [42]:
bostonPD_by_year

year
2016   388587231.110
2017   391277619.770
2018   416682999.940
2019   415725606.800
Name: TOTAL EARNINGS, dtype: float64

In [80]:
import sys
import os
sys.path.insert(0, "/pensions")
from LocalPD_True_Payroll import Boston_total_earnings


In [86]:
boston_total_no_teachers, boston_total = Boston_total_earnings()

WARNING! IN DEBUG MODE


In [85]:
boston_total.loc[boston_total["department"] == "Boston Police Department", "TOTAL EARNINGS"].sum()/\
boston_total.loc[:,"TOTAL EARNINGS"].sum()

0.24105440360836722

In [88]:
boston_total_no_teachers.loc[boston_total_no_teachers["department"] == "Boston Police Department", "TOTAL EARNINGS"].sum()/\
boston_total_no_teachers.loc[:,"TOTAL EARNINGS"].sum()

0.4289951144834596